The provided file has been slightly changed by me as per requirement. \\
new file has been attached

In [ ]:
#installing required packages
!pip install -q pyomo
from pyomo.environ import *
import numpy as np
import pandas as pd

#importing data
data = pd.read_csv('pq.csv')
data_main=data.copy()

#creating a list of route names
listm=[list(data.columns).copy()]
del listm[0][0]

#creating another list of route names
h_list=list(data_main.columns)
h_list.remove('route')

In [ ]:
#a function is defined, it makes two groups one of the routes which intersect with route in 1st row and other of those who do not.
#it also make the diagonal entries 1, for convinience.
#for example when it is used once it, removes the rows and columns of those routes which do not intersect with route in 1st row.
#So, using it multiple time we can group those routes which intersect with M1 in min. no. of groups as required.

def min_colour_fn(df):
  h_list=list(df.columns)
  i=1
  while i < len(h_list):
    df.loc[i-1, h_list[i]] = 0
    i=i+1

  list1=[]
  dlist=[]
  global listm
  for i in df.columns:
    if df[i][0]==1:
      list1.append(h_list.index(i))
      for lst in listm:
        if i in lst:
          lst.remove(i)
      dlist.append(i)
  listm.append(dlist)

  list2=[]
  for i in df.columns:
    if h_list.index(i)!=0:
      if df[i][0]!=1:
        list2.append(i)

  if len(list1)>=1:
    i1=1
    df.drop(list2,axis=1,inplace=True)
    row_drop_list=[x for x in range(len(h_list)-1) if x+1 not in list1]
    df=df.drop(row_drop_list,inplace=True)
  else:
    i1=0
    df.drop(list2[1],axis=1,inplace=True)
    df.drop(0,axis=0,inplace=True)

  return df,i1

In [ ]:
#we are using above function multiple times so as to get optimal number of groups for routes with which M1 intersects
#after using this the routes(including M1) with which M1 do not intersects are still grouped together. They will be dealt by next function.
k=1
while data.shape[0]>1:
  if min_colour_fn(data)[1]==1:
    k=k+1
  else:
    k=k
  data.to_csv('data.csv')
  data=pd.read_csv('data.csv').drop('Unnamed: 0',axis=1)

In [ ]:
#this function takes those elements which cannot be present in group of non intersecting routes(with M1) and tries to accomodate in already present groups
#in listm(with the help of given intersection data). If a route from group of non intersecting routes(with M1) caannot be accomodated with any of already 
#present groups than another group is formed.
def fn():
  global listm
  for i in listm[0]:
    for j in listm[0]:
      if h_list.index(i)<h_list.index(j):
        if data_main[j][h_list.index(i)]==1:
          listm[0].remove(j)
          i2=1
          for k in listm[1:]:
            if i2==1:
              i1=1
              for p in k:
                if h_list.index(j)<h_list.index(p):
                  if data_main[p][h_list.index(j)]==1:
                    i1=0
                elif h_list.index(j)>h_list.index(p):
                  if data_main[j][h_list.index(p)]==1:
                    i1=0
              if i1==1:
                k.append(j)
                i2=0
          if i2==1:
            listm.append([j])
  return listm

In [ ]:
#We will stop using just above function when their is no change in 'listm'
#and the routes within a list in listm can be colored with same colour
#minimum no. of colour which can be used will be the length of listm after this code
listM=listm.copy()
while listM != fn():
  listM=listm.copy()
  fn()

In [ ]:
listm

[['M1', 'M2', 'M17', '61'],
 ['M5', '12', '50', '18', '21', '62'],
 ['M8', '60'],
 ['M10', '16', '67'],
 ['M13', '27', '68'],
 ['M4', '37', '63'],
 ['M6']]

In [ ]:
len(listm)

7

In [ ]:
for i in listm:
  print(i)

['M1', 'M2', 'M17', '61']
['M5', '12', '50', '18', '21', '62']
['M8', '60']
['M10', '16', '67']
['M13', '27', '68']
['M4', '37', '63']
['M6']


The above result show that The routes in a group can be coloured same and a group should have different colour than other groups.

Hence, 7 is the possible minimum number of colours which can be used.